In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

Using TensorFlow backend.


## Data preparation

In [2]:
path = '/axp/rim/imsads/dev/sagra39/DigitRecognizer/'
train = pd.read_csv(path + "train.csv")
test= pd.read_csv(path + "test.csv")
sample_submission= pd.read_csv(path + "sample_submission.csv")

In [3]:
X_train = (train.ix[:,1:].values).astype('float32') # all pixel values
y_train = train.ix[:,0].values.astype('int32') # only labels i.e targets digits
X_test_new = test.values.astype('float32')

In [4]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train.shape
X_test_new = X_test_new.reshape(X_test_new.shape[0], 28, 28, 1)
X_test_new.shape

(28000, 28, 28, 1)

In [5]:
X_train = X_train.astype('float32')
X_test_new = X_test_new.astype('float32')
X_train /= 255
X_test_new /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test_new.shape[0], 'test samples')

x_train shape: (42000, 28, 28, 1)
42000 train samples
28000 test samples


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train  , y_test  = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
print(X_train.shape[0], 'train samples')
print(X_test_new.shape[0], 'test samples')

42000 train samples
28000 test samples


In [8]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
batch_size = 32
epochs = 100
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

## Data Augmentation

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        25632     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          51264     
__________

In [243]:
import time
start_time = time.time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_data=(x_test, y_test))
print (time.time() - start_time)

3944.809206008911


In [244]:
score1 = model.predict(X_test_new)

In [245]:
for index, row in sample_submission.iterrows():
    row['Label'] = np.argmax(score1[index])
    
print (sample_submission.Label.nunique())
sample_submission.to_csv(path + 'final4.csv', index=False)

10
